#  TCP Traffic
<i>Adapted for use with FABRIC from [TCP Traffic](https://www.cs.unc.edu/Research/geni/geniEdu/03-TcpTraffic.html) and from [TCP vs. UDP](https://www.cs.unc.edu/Research/geni/geniEdu/07-TCPvsUDP.html) </i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This experiment will provide you with the experience of generating and analyzing TCP flows using iperf. This experiment will also demonstrate the differences in UDP and TCP by exploring how the two protocols share the link when running simultaneously. You will use iperf to create the flow from a node to another and see how congestion control reacts once UDP traffic has started

This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

In the first section the module gives students experience generating and analyzing TCP flows. Students will use iperf to create a flow and view the sawtooth behavior. A second flow will then be introduced to show how TCP flows share a link. (Iperf)

In the second section the module demonstrates differences in UDP and TCP by exploring how the two protocols share the link when running simultaneously. Students will use iperf to create a TCP flow from one node to another and UDP to create a flow in the other direction to show how TCP's congestion control reacts to UDP traffic.

## 1. Setup the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'TCP_Traffic_intro'
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    slice.list_nodes()
    
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Guided Experiment
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/TCP%20Traffic/figures/TCPTrafficTop.png?raw=true"><br>
### 2.1 Iperf in one direction
iperf is a tool for measuring TCP and UDP bandwidth performance. In this section we will measure from client to server.
1. In the Server SSH terminal, start the iperf with the following command: ```iperf -s``` 
2. In the Client SSH terminal run the iperf command by typing ```iperf -c <nodeA_ip> -i 10 -t 180```
<br> The "-s" indicates that the node will run in server mode. Likewise the "-c" is client mode. The "-i 10" tells iperf to print updates every 10 seconds, and "-t 180" to run for 180 before closing the connection.
<br> Running iperf can be done in either direction. You would just need to use the commands on the opposite terminals with the correct ip for the second command. You can always find the ip in linux using either "ifconfig" or "ip a". The ip for your node in the network should be under "ens#"

### 2.2 TCP vs UDP
In this exercise we will see the link that the TCP and UDP traffic share when they are running simultaneously. We will use iperf to generate the traffic between the nodes and see how the congestion control will affect the output.
1. Start 2 terminals on node A and a two terminals on node B
2. First we will start the servers on the first terminal of node A. Run the following command to start the UDP server and the TCP servers: ```iperf -s -i 15 & iperf -s -u -i 10```. The TCP server will print information every 15 seconds, and the UDP server will print information every 10 seconds.
3. On the second A terminal prepare to run the script: ```./ss-output.sh```. This will catch packets so that you are able to analyze the traffic changes in this experiment.
4. Start the script on terminal A and then immediately run the following command: ```iperf -c <nodeA> -t 90 ```. On the first node B terminal, this start the tcp trafic and will run for 90 seconds (1.5 minutes).
6. After 30 seconds run the following command: ```iperf -c <NodeA> -t 30 -u -b 200m```. On the second B terminal this will start sending UDP traffic at the same time as TCP traffic. This will run for a minute.
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/TCP%20Traffic/figures/TUT.png?raw=true"><br>
7. Once the traffic stops for both nodes stop the command on the second A terminal by hitting ```Ctrl-C```. This command should take about 3-5 more minutes depending on the hardware and site you used.


In [ ]:
# Download chart from Node A
try:
    A = slice.get_node('ND_A')
    A.execute("Rscript ss-data-analysis.R  ")
    A.download_file("Traffic-ss.svg","Traffic-ss.svg")
except Exception as e:
    print(f"Fail: {e}")

See the output [here](./Traffic-ss.svg)

9. Now that you have seen the traffic rerun the experiment steps 2 - 8 with step 6's command modified to have increased bandwidth: ```iperf -c <client> -t 30 -u -b 500m```
10. Run the following cell to compare the traffic.

In [ ]:
# Download second chart from Node A
try:
    A.execute("Rscript ss-data-analysis.R  ")
    A.download_file("Traffic2.svg","Traffic-ss.svg")
except Exception as e:
    print(f"Fail: {e}")

See the output [here](./Traffic2.svg)

## 3. Exploration

Set up the servers once again and start TCP traffic from client to the server VM.
1. Report the bandwidth output of the TCP diagnosis (on the client). It should look like
<br> `[ ID] Interval   	Transfer 	Bandwidth`
<br> `[  3]  0.0-60.0 sec   683 MBytes  95.5 Mbits/sec`

2. Alter the UDP bandwidth with the -b option: what is the smallest UDP bandwidth at which you can see the TCP graph react?
3. Starting TCP and UDP traffic at the same time with -t option: at what UDP -b option do TCP and UDP to fairly share the link? That is, at what point does the TCP bandwidth approximate the UDP bandwidth? (Use the TCP client's output and an estimation of the UDP's average bandwidth.)
4. Report how the sum of the shared bandwidths in part 3 compares to the bandwidth of TCP in part 1 when it is not sharing the link.


## Cleanup Resources
Once you have completed the assignment shut down the slice.

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")